# Task 4.3.5: Drift Detection with Evidently AI

**Module:** 4.3 - MLOps & Experiment Tracking  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the types of drift that affect ML models
- [ ] Set up Evidently AI for model monitoring
- [ ] Detect data drift and concept drift in production
- [ ] Create monitoring dashboards and alerts
- [ ] Build a drift monitoring pipeline

---

## Prerequisites

- Completed: Task 4.3.4 (Custom Evaluation)
- Knowledge of: Basic statistics, model deployment concepts

---

## Real-World Context

Your model performed great during testing. You deployed it. Two months later, accuracy tanks. What happened?

**Drift** happened.

Real examples:
- **COVID-19**: Fraud detection models failed when shopping patterns changed overnight
- **Seasonal shifts**: Recommendation systems degrade when user preferences change
- **Data pipeline issues**: A sensor starts returning values in different units

Companies like Uber, Netflix, and Airbnb continuously monitor for drift because catching it early saves millions.

---

## ELI5: What is Drift?

> **Imagine you learned to drive in sunny California.**
>
> You're great at it! Clear roads, perfect weather, easy traffic.
>
> Then you move to Minnesota in winter. Suddenly:
> - The roads are icy (different input data)
> - Driving rules are different (different relationships)
> - Your skills don't work as well (degraded performance)
>
> **In AI terms:** Drift is when the real-world data or relationships change compared to what your model was trained on. There are two main types:
> - **Data drift**: The inputs look different (California → Minnesota weather)
> - **Concept drift**: The rules changed (summer driving → winter driving)

---

## Types of Drift

| Type | What Changes | Example | Detection |
|------|-------------|---------|----------|
| **Data Drift** | Input distribution | Users are younger now | Compare feature distributions |
| **Concept Drift** | Input→Output relationship | "Spam" definition changed | Monitor prediction quality |
| **Label Drift** | Output distribution | More positive reviews | Compare prediction distributions |
| **Upstream Drift** | Data pipeline | New data format | Monitor data quality |

---

## Part 1: Setting Up Evidently AI

Evidently AI is an open-source tool for ML monitoring and testing.

In [ ]:
# Install Evidently
!pip install evidently -q

import evidently
print(f"Evidently version: {evidently.__version__}")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Evidently imports
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset
from evidently.metrics import (
    DataDriftTable,
    DatasetDriftMetric,
    ColumnDriftMetric,
    DatasetSummaryMetric
)

print("Imports ready!")

---

## Part 2: Creating Sample Data

Let's simulate a real scenario: a sentiment classifier deployed in production.

In [ ]:
def generate_training_data(n_samples: int = 1000, seed: int = 42) -> pd.DataFrame:
    """
    Generate synthetic training data for a sentiment classifier.
    
    Features represent text statistics like:
    - text_length: Number of characters
    - word_count: Number of words
    - avg_word_length: Average word length
    - sentiment_keywords: Count of sentiment-related words
    - exclamation_count: Number of exclamation marks
    """
    np.random.seed(seed)
    
    data = {
        'text_length': np.random.normal(150, 50, n_samples).clip(20, 500),
        'word_count': np.random.normal(30, 10, n_samples).clip(5, 100),
        'avg_word_length': np.random.normal(5.5, 1.0, n_samples).clip(3, 10),
        'sentiment_keywords': np.random.poisson(3, n_samples),
        'exclamation_count': np.random.poisson(1, n_samples),
        'question_marks': np.random.poisson(0.5, n_samples),
        'uppercase_ratio': np.random.beta(2, 20, n_samples),
    }
    
    df = pd.DataFrame(data)
    
    # Generate target based on features (simulated relationship)
    score = (
        0.3 * (df['sentiment_keywords'] > 2) +
        0.2 * (df['exclamation_count'] > 0) +
        0.2 * (df['text_length'] > 100) +
        0.3 * np.random.random(n_samples)
    )
    df['target'] = (score > 0.5).astype(int)  # Binary: positive/negative
    df['prediction'] = df['target']  # Perfect predictions during training
    
    return df

# Generate reference (training) data
reference_data = generate_training_data(n_samples=2000)

print("Reference (Training) Data:")
print(f"  Shape: {reference_data.shape}")
print(f"  Target distribution: {reference_data['target'].value_counts().to_dict()}")
reference_data.head()

In [ ]:
def generate_production_data(
    n_samples: int = 500,
    drift_type: str = "none",
    drift_intensity: float = 0.5,
    seed: int = 123
) -> pd.DataFrame:
    """
    Generate production data with optional drift.
    
    Args:
        n_samples: Number of samples
        drift_type: "none", "data_drift", "concept_drift", "both"
        drift_intensity: How severe the drift is (0-1)
        seed: Random seed
    """
    np.random.seed(seed)
    
    # Start with similar distribution to training
    data = {
        'text_length': np.random.normal(150, 50, n_samples).clip(20, 500),
        'word_count': np.random.normal(30, 10, n_samples).clip(5, 100),
        'avg_word_length': np.random.normal(5.5, 1.0, n_samples).clip(3, 10),
        'sentiment_keywords': np.random.poisson(3, n_samples),
        'exclamation_count': np.random.poisson(1, n_samples),
        'question_marks': np.random.poisson(0.5, n_samples),
        'uppercase_ratio': np.random.beta(2, 20, n_samples),
    }
    
    df = pd.DataFrame(data)
    
    # Apply data drift
    if drift_type in ["data_drift", "both"]:
        # Shift distributions
        df['text_length'] = df['text_length'] * (1 + drift_intensity * 0.5)  # Longer texts
        df['exclamation_count'] = np.random.poisson(1 + 3 * drift_intensity, n_samples)  # More excited
        df['uppercase_ratio'] = np.random.beta(2 + 5 * drift_intensity, 20, n_samples)  # More caps
    
    # Generate predictions (model's behavior)
    score = (
        0.3 * (df['sentiment_keywords'] > 2) +
        0.2 * (df['exclamation_count'] > 0) +
        0.2 * (df['text_length'] > 100) +
        0.3 * np.random.random(n_samples)
    )
    df['prediction'] = (score > 0.5).astype(int)
    
    # True labels (may differ if concept drift)
    if drift_type in ["concept_drift", "both"]:
        # The relationship between features and target has changed
        # Now uppercase_ratio matters more
        new_score = (
            0.1 * (df['sentiment_keywords'] > 2) +  # Less important now
            0.1 * (df['exclamation_count'] > 0) +   # Less important now
            0.4 * (df['uppercase_ratio'] > 0.1) +   # Now more important!
            0.4 * np.random.random(n_samples)
        )
        df['target'] = (new_score > 0.5).astype(int)
    else:
        df['target'] = df['prediction']  # No concept drift means good predictions
    
    # Add some noise to predictions
    noise_mask = np.random.random(n_samples) < (0.1 + drift_intensity * 0.2)
    df.loc[noise_mask, 'prediction'] = 1 - df.loc[noise_mask, 'prediction']
    
    return df

# Generate production data scenarios
production_no_drift = generate_production_data(drift_type="none")
production_data_drift = generate_production_data(drift_type="data_drift", drift_intensity=0.8)
production_concept_drift = generate_production_data(drift_type="concept_drift", drift_intensity=0.7)
production_both = generate_production_data(drift_type="both", drift_intensity=0.6)

print("Production Data Scenarios Generated:")
print(f"  No drift: {len(production_no_drift)} samples")
print(f"  Data drift: {len(production_data_drift)} samples")
print(f"  Concept drift: {len(production_concept_drift)} samples")
print(f"  Both: {len(production_both)} samples")

---

## Part 3: Detecting Data Drift

Let's use Evidently to compare the production data to our reference data.

In [ ]:
# Define column mapping
column_mapping = ColumnMapping(
    target='target',
    prediction='prediction',
    numerical_features=[
        'text_length', 'word_count', 'avg_word_length',
        'sentiment_keywords', 'exclamation_count',
        'question_marks', 'uppercase_ratio'
    ]
)

print("Column mapping configured!")

In [ ]:
def run_drift_report(
    reference: pd.DataFrame,
    current: pd.DataFrame,
    column_mapping: ColumnMapping,
    report_name: str = "drift_report"
) -> Report:
    """
    Generate a comprehensive drift report.
    """
    report = Report(metrics=[
        DatasetDriftMetric(),
        DataDriftTable(),
    ])
    
    report.run(
        reference_data=reference,
        current_data=current,
        column_mapping=column_mapping
    )
    
    return report

# Test with no-drift scenario
report_no_drift = run_drift_report(
    reference_data, 
    production_no_drift, 
    column_mapping
)

print("No-drift report generated!")

In [ ]:
# Extract drift results
def extract_drift_summary(report: Report) -> dict:
    """Extract key drift metrics from report."""
    results = report.as_dict()
    
    summary = {}
    
    for metric in results.get('metrics', []):
        metric_id = metric.get('metric', '')
        result = metric.get('result', {})
        
        if 'DatasetDriftMetric' in metric_id:
            summary['dataset_drift'] = result.get('dataset_drift', False)
            summary['drift_share'] = result.get('drift_share', 0)
            summary['n_drifted_features'] = result.get('number_of_drifted_columns', 0)
        
        if 'DataDriftTable' in metric_id:
            summary['column_drifts'] = {}
            for col_data in result.get('drift_by_columns', {}).values():
                col_name = col_data.get('column_name', 'unknown')
                summary['column_drifts'][col_name] = {
                    'drifted': col_data.get('drift_detected', False),
                    'drift_score': col_data.get('drift_score', 0),
                    'stattest': col_data.get('stattest_name', 'unknown')
                }
    
    return summary

# Check no-drift scenario
summary_no_drift = extract_drift_summary(report_no_drift)
print("No-Drift Scenario:")
print(f"  Dataset drift detected: {summary_no_drift.get('dataset_drift', 'N/A')}")
print(f"  Drift share: {summary_no_drift.get('drift_share', 0):.2%}")

In [ ]:
# Now check data drift scenario
report_data_drift = run_drift_report(
    reference_data,
    production_data_drift,
    column_mapping
)

summary_data_drift = extract_drift_summary(report_data_drift)

print("Data Drift Scenario:")
print(f"  Dataset drift detected: {summary_data_drift.get('dataset_drift', 'N/A')}")
print(f"  Drift share: {summary_data_drift.get('drift_share', 0):.2%}")
print(f"  Features drifted: {summary_data_drift.get('n_drifted_features', 0)}")

print("\nPer-column drift:")
for col, info in summary_data_drift.get('column_drifts', {}).items():
    if col in ['target', 'prediction']:
        continue
    status = "DRIFTED" if info['drifted'] else "OK"
    print(f"  {col}: {status} (score: {info['drift_score']:.4f})")

In [ ]:
# Save HTML report
report_data_drift.save_html("data_drift_report.html")
print("Report saved to data_drift_report.html")
print("Open in browser to see interactive visualizations!")

### What Just Happened?

Evidently compared the statistical distributions of each feature:
- Used statistical tests (Kolmogorov-Smirnov for numerical features)
- Calculated drift scores for each column
- Flagged columns where the distribution significantly changed

---

## Part 4: Data Quality Monitoring

Beyond drift, we should monitor data quality issues.

In [ ]:
from evidently.metrics import (
    DatasetMissingValuesMetric,
    ColumnQuantileMetric,
    ColumnValueRangeMetric
)

def run_quality_report(
    reference: pd.DataFrame,
    current: pd.DataFrame,
    column_mapping: ColumnMapping
) -> Report:
    """
    Generate a data quality report.
    """
    report = Report(metrics=[
        DatasetSummaryMetric(),
        DatasetMissingValuesMetric(),
    ])
    
    report.run(
        reference_data=reference,
        current_data=current,
        column_mapping=column_mapping
    )
    
    return report

In [ ]:
# Create data with quality issues
production_bad_quality = production_no_drift.copy()

# Introduce missing values
production_bad_quality.loc[
    np.random.random(len(production_bad_quality)) < 0.1, 
    'sentiment_keywords'
] = np.nan

# Introduce outliers
production_bad_quality.loc[
    np.random.random(len(production_bad_quality)) < 0.05,
    'text_length'
] = 10000  # Unreasonably long

print("Introduced quality issues:")
print(f"  Missing values: {production_bad_quality['sentiment_keywords'].isna().sum()}")
print(f"  Outliers (text_length > 1000): {(production_bad_quality['text_length'] > 1000).sum()}")

In [ ]:
# Run quality report
quality_report = run_quality_report(
    reference_data,
    production_bad_quality,
    column_mapping
)

quality_report.save_html("data_quality_report.html")
print("Quality report saved!")

---

## Part 5: Model Performance Monitoring

Track how model performance changes over time.

In [ ]:
from evidently.metric_preset import ClassificationPreset
from evidently.metrics import (
    ClassificationQualityMetric,
    ClassificationClassBalance,
    ClassificationConfusionMatrix
)

def run_performance_report(
    reference: pd.DataFrame,
    current: pd.DataFrame,
    column_mapping: ColumnMapping
) -> Report:
    """
    Monitor classification model performance.
    """
    report = Report(metrics=[
        ClassificationQualityMetric(),
        ClassificationClassBalance(),
        ClassificationConfusionMatrix()
    ])
    
    report.run(
        reference_data=reference,
        current_data=current,
        column_mapping=column_mapping
    )
    
    return report

In [ ]:
# Compare performance: no drift vs concept drift
print("Performance with NO drift:")
perf_no_drift = run_performance_report(reference_data, production_no_drift, column_mapping)
perf_results_no_drift = perf_no_drift.as_dict()

# Extract accuracy
for metric in perf_results_no_drift.get('metrics', []):
    if 'ClassificationQuality' in metric.get('metric', ''):
        result = metric.get('result', {}).get('current', {})
        print(f"  Accuracy: {result.get('accuracy', 'N/A'):.4f}")
        print(f"  Precision: {result.get('precision', 'N/A'):.4f}")
        print(f"  Recall: {result.get('recall', 'N/A'):.4f}")
        print(f"  F1: {result.get('f1', 'N/A'):.4f}")

In [ ]:
print("\nPerformance with CONCEPT drift:")
perf_concept_drift = run_performance_report(reference_data, production_concept_drift, column_mapping)
perf_results_concept = perf_concept_drift.as_dict()

for metric in perf_results_concept.get('metrics', []):
    if 'ClassificationQuality' in metric.get('metric', ''):
        result = metric.get('result', {}).get('current', {})
        print(f"  Accuracy: {result.get('accuracy', 'N/A'):.4f}")
        print(f"  Precision: {result.get('precision', 'N/A'):.4f}")
        print(f"  Recall: {result.get('recall', 'N/A'):.4f}")
        print(f"  F1: {result.get('f1', 'N/A'):.4f}")

print("\nNotice the performance degradation due to concept drift!")

---

## Part 6: Building a Monitoring Pipeline

Let's create a reusable monitoring pipeline.

In [ ]:
from typing import Optional, List
from dataclasses import dataclass
import json

@dataclass
class MonitoringAlert:
    """Alert generated by monitoring."""
    severity: str  # "info", "warning", "critical"
    category: str  # "data_drift", "concept_drift", "quality", "performance"
    message: str
    details: dict


class ModelMonitor:
    """
    Comprehensive model monitoring system.
    
    Example:
        monitor = ModelMonitor(reference_data, column_mapping)
        alerts = monitor.check(current_data)
        for alert in alerts:
            print(f"{alert.severity}: {alert.message}")
    """
    
    def __init__(
        self,
        reference_data: pd.DataFrame,
        column_mapping: ColumnMapping,
        drift_threshold: float = 0.3,
        accuracy_threshold: float = 0.8,
        missing_threshold: float = 0.05
    ):
        self.reference_data = reference_data
        self.column_mapping = column_mapping
        self.drift_threshold = drift_threshold
        self.accuracy_threshold = accuracy_threshold
        self.missing_threshold = missing_threshold
        self.history: List[dict] = []
    
    def check(self, current_data: pd.DataFrame) -> List[MonitoringAlert]:
        """
        Run all monitoring checks and return alerts.
        """
        alerts = []
        check_results = {
            "timestamp": datetime.now().isoformat(),
            "n_samples": len(current_data)
        }
        
        # Check data drift
        drift_alerts, drift_results = self._check_data_drift(current_data)
        alerts.extend(drift_alerts)
        check_results["drift"] = drift_results
        
        # Check data quality
        quality_alerts, quality_results = self._check_data_quality(current_data)
        alerts.extend(quality_alerts)
        check_results["quality"] = quality_results
        
        # Check performance (if labels available)
        if 'target' in current_data.columns:
            perf_alerts, perf_results = self._check_performance(current_data)
            alerts.extend(perf_alerts)
            check_results["performance"] = perf_results
        
        self.history.append(check_results)
        return alerts
    
    def _check_data_drift(self, current_data: pd.DataFrame) -> tuple:
        """Check for data drift."""
        alerts = []
        
        report = Report(metrics=[DatasetDriftMetric(), DataDriftTable()])
        report.run(
            reference_data=self.reference_data,
            current_data=current_data,
            column_mapping=self.column_mapping
        )
        
        results = report.as_dict()
        drift_share = 0
        drifted_columns = []
        
        for metric in results.get('metrics', []):
            if 'DatasetDriftMetric' in metric.get('metric', ''):
                result = metric.get('result', {})
                drift_share = result.get('drift_share', 0)
                
            if 'DataDriftTable' in metric.get('metric', ''):
                for col_data in metric.get('result', {}).get('drift_by_columns', {}).values():
                    if col_data.get('drift_detected', False):
                        drifted_columns.append(col_data.get('column_name', 'unknown'))
        
        if drift_share > self.drift_threshold:
            severity = "critical" if drift_share > 0.5 else "warning"
            alerts.append(MonitoringAlert(
                severity=severity,
                category="data_drift",
                message=f"Data drift detected in {drift_share:.1%} of features",
                details={"drift_share": drift_share, "drifted_columns": drifted_columns}
            ))
        
        return alerts, {"drift_share": drift_share, "drifted_columns": drifted_columns}
    
    def _check_data_quality(self, current_data: pd.DataFrame) -> tuple:
        """Check data quality."""
        alerts = []
        
        # Check missing values
        missing_ratio = current_data.isnull().mean().mean()
        
        if missing_ratio > self.missing_threshold:
            alerts.append(MonitoringAlert(
                severity="warning",
                category="quality",
                message=f"High missing value ratio: {missing_ratio:.1%}",
                details={"missing_ratio": missing_ratio}
            ))
        
        return alerts, {"missing_ratio": missing_ratio}
    
    def _check_performance(self, current_data: pd.DataFrame) -> tuple:
        """Check model performance."""
        alerts = []
        
        # Calculate accuracy
        accuracy = (current_data['prediction'] == current_data['target']).mean()
        
        if accuracy < self.accuracy_threshold:
            severity = "critical" if accuracy < 0.6 else "warning"
            alerts.append(MonitoringAlert(
                severity=severity,
                category="performance",
                message=f"Model accuracy dropped to {accuracy:.1%}",
                details={"accuracy": accuracy, "threshold": self.accuracy_threshold}
            ))
        
        return alerts, {"accuracy": accuracy}
    
    def get_summary(self) -> pd.DataFrame:
        """Get historical monitoring summary."""
        if not self.history:
            return pd.DataFrame()
        
        rows = []
        for record in self.history:
            row = {
                "timestamp": record["timestamp"],
                "n_samples": record["n_samples"],
                "drift_share": record.get("drift", {}).get("drift_share", 0),
                "missing_ratio": record.get("quality", {}).get("missing_ratio", 0),
                "accuracy": record.get("performance", {}).get("accuracy", None)
            }
            rows.append(row)
        
        return pd.DataFrame(rows)

In [ ]:
# Test the monitoring pipeline
monitor = ModelMonitor(
    reference_data=reference_data,
    column_mapping=column_mapping,
    drift_threshold=0.2,
    accuracy_threshold=0.85
)

# Check various scenarios
scenarios = [
    ("No drift", production_no_drift),
    ("Data drift", production_data_drift),
    ("Concept drift", production_concept_drift),
    ("Both types", production_both)
]

for name, data in scenarios:
    print(f"\n{'='*50}")
    print(f"Scenario: {name}")
    print(f"{'='*50}")
    
    alerts = monitor.check(data)
    
    if alerts:
        for alert in alerts:
            icon = {"critical": "🔴", "warning": "🟡", "info": "🔵"}[alert.severity]
            print(f"  {icon} [{alert.severity.upper()}] {alert.message}")
    else:
        print("  ✅ All checks passed!")

In [ ]:
# View monitoring history
summary = monitor.get_summary()
print("\nMonitoring History:")
print(summary.to_string())

---

## Try It Yourself

Create a monitoring dashboard that:
1. Simulates data arriving over time (with increasing drift)
2. Runs checks at regular intervals
3. Plots drift metrics over time

<details>
<summary>Hint</summary>

Use a loop to simulate time periods:
```python
for day in range(30):
    drift_intensity = day / 30  # Increasing drift
    data = generate_production_data(drift_intensity=drift_intensity)
    alerts = monitor.check(data)
```

</details>

In [ ]:
# YOUR CODE HERE
# Simulate drift over time and visualize

# Your monitoring simulation...

---

## Common Mistakes

### Mistake 1: Not Having Good Reference Data

```python
# Wrong - using biased or small reference
reference = training_data.sample(100)  # Too small!

# Right - use representative reference data
reference = training_data.sample(min(10000, len(training_data)))
# Or use a dedicated holdout set
```
**Why:** Small or biased reference data leads to false drift alarms.

### Mistake 2: Alerting on Every Drift

```python
# Wrong - alert on any statistical difference
if p_value < 0.05:
    send_alert("DRIFT DETECTED!")  # Too sensitive

# Right - set practical thresholds
if drift_share > 0.3 and effect_size > 0.2:
    send_alert("Significant drift detected")
```
**Why:** Small statistical differences are normal. Alert on meaningful changes.

### Mistake 3: Ignoring Concept Drift

```python
# Wrong - only monitoring inputs
check_data_drift(inputs)

# Right - also monitor performance
check_data_drift(inputs)
check_prediction_distribution(predictions)
check_actual_performance(predictions, delayed_labels)
```
**Why:** Concept drift (changed relationships) won't show in input distributions.

### Mistake 4: Not Automating Monitoring

```python
# Wrong - manual checks
# "I'll run the drift check next week..."

# Right - automated pipeline
# Schedule with cron, Airflow, or similar
@scheduled(cron="0 0 * * *")  # Daily at midnight
def run_monitoring():
    data = get_last_24h_data()
    alerts = monitor.check(data)
    if alerts:
        send_to_slack(alerts)
```
**Why:** Manual monitoring gets forgotten. Automate it.

---

## Checkpoint

You've learned:
- What data drift and concept drift are
- How to use Evidently AI for drift detection
- How to monitor data quality in production
- How to track model performance over time
- How to build a monitoring pipeline with alerts

---

## Challenge (Optional)

Build a complete monitoring system that:
1. Stores monitoring results in MLflow
2. Creates a live dashboard using Streamlit or Gradio
3. Sends Slack/email alerts when thresholds are exceeded
4. Automatically triggers model retraining on severe drift

---

## Further Reading

- [Evidently AI Documentation](https://docs.evidentlyai.com/)
- [Made with ML: Monitoring](https://madewithml.com/courses/mlops/monitoring/)
- [Google MLOps: Monitoring](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)

---

## Cleanup

In [ ]:
# Clean up
import os

# Remove generated reports
for f in ["data_drift_report.html", "data_quality_report.html"]:
    if os.path.exists(f):
        os.remove(f)

print("Cleanup complete!")

---

## Next Steps

Now that you can detect when your model needs updating, the next notebook covers **model versioning** - how to manage multiple model versions and safely roll back if needed.

**Continue to:** [06-model-registry.ipynb](06-model-registry.ipynb)